<center>

## Live demo
<img src="pathpy_logo.png" alt="pathpy" style="width: 400px;"/>
</center>

<div class="alert alert-danger">
In this tutorial the alpha version of pathpy3 is used. With high probability there will be code changes! To complete this demo successfully use pathpy version v3.0.0a2!
</div>

**Jürgen Hackl**  
Assistant Professor  
School of Engineering  
University of Liverpool, UK   

**September 17 2020**

# Table of Content
**1 [Introducing `pathpy`](#part-1)**

1.1 [Creating networks](#part-1-1)  
1.2 [Node and Edge objects](#part-1-2)  
1.3 [Attributes](#part-1-3)  
1.4 [Functions](#part-1-4)

**2 [Visualising networks with `pathpy`](#part-2)**

2.1 [Interactive network visualisation in `jupyter`](#part-2-1)  
2.2 [Network Layouts](#part-2-2)  
2.3 [Plotting spatially embedded networks](#part-2-3)  
2.4 [Plotting networks to HTML, PDF and LaTeX](#part-2-4)

**3 [Temporal network analysis and visualisation in `pathpy`](#part-3)**

3.1 [Creating temporal networks](#part-3-1)  
3.2 [Calculating causal paths](#part-3-2)  
3.3 [Time-aggregated networks](#part-3-3)  
3.4 [Visualization of temporal networks](#part-3-4)

**4 [Higher-order models of paths](#part-4)**

4.1 [Path and PathCollection](#part-4-1)  
4.2 [Creating higher-order networks](#part-4-2)  
4.3 [Ranking nodes in higher-order networks](#part-4-3)  
4.4 [Multi-order model selection](#part-4-4)

**5 [Outlook](#part-5)**

5.1 [Hypergraphs](#part-5-1)  
5.2 [Milestones](#part-5-2)  
5.3 [Ways to contribute](#part-5-3)  

# 1 Introducing `pathpy`<a class="anchor" id="part-1"></a>

1.1 [Creating networks](#part-1-1)  
1.2 [Node and Edge objects](#part-1-2)  
1.3 [Attributes](#part-1-3)  
1.4 [Functions](#part-1-4)

Compared to many other packages, `pathpy` has a couple of advantages. First, it is easy to install as should have no dependencies not included in a default `anaconda` installation. Second, `pathpy` has a user-friendly API that makes it easy to handle directed and undirected networks, networks with single and multiple edges, multi-layer networks or temporal networks. It also provides interactive HTML visualisations that can be directly displayed inside `jupyter` notebooks, which makes it particularly suitable for educational settings. Third, it supports the analysis and visualisation of time series data on networked systems, such as time-stamped edges or data on paths in networks.

We first import `pathpy` and assign the local alias `pp`:

In [ ]:
import pathpy as pp
pp.__version__

If the `import` statement completes without error message, the installation was successful and we can now use `pathpy` to generate, analyse, and visualise networks. 

## 1.1 Creating networks <a class="anchor" id="part-1-1"></a>

For this purpose `pathpy` provides the `Network` class. Calling the constructor will return an instance that represents an empty network with no nodes and no links. By default, networks in `pathpy` are directed. If you want to create an undirected network you can pass the parameter `directed=False` in the constructor. 

Printing the `Network` object will give a short string summary which tells whether the network is directed or undirected, as well as the number of unique nodes and links.

In [ ]:
n1 = pp.Network()
print(n1)

A network is directed by default, but we can create an undirected network by passing the parameter `directed=False`.

In [ ]:
n2 = pp.Network(directed=False)
print(n2)

The examples above show that each network has a unique identifier. By default, this unique ID is derived from the hash value of the underlying python object, which allows you to quickly check whether two variables actually refer to the same network object. If you prefer to manage your own UIDs that are easier to remember, you can assign custom IDs by explicitly passing a uid property:

In [ ]:
n = pp.Network(directed=False, uid='MyUndirectedNetwork')
print(n)

The simplest way to add nodes and edges is to call the functions `add_node` and `add_edge`. In both cases, we can simply pass unique string identifiers of nodes, which will then be used as UIDs of the underlying node objects. To create an undirected network with three nodes and two edges, we can write: 

In [ ]:
n = pp.Network(directed=False, uid='ExampleNetwork')
n.add_node('a')
n.add_node('b')
n.add_node('c')
n.add_edge('a', 'b')
n.add_edge('b', 'c')
print(n)

In [ ]:
n

Unless we want to explicitly add isolated nodes with no incident edges, we can omit the explicit call of the `add_node` function. If we add edges any node that does not exist already will be created and added automatically. If we want to check explicitly whether a node exists or node before creating and edge, we can test this with the `in` operator on the set of node available via `Network.nodes`:

In [ ]:
'd' in n.nodes

The following code will automatically add a new node `d`, along with a new edge from `d` to `a`.

In [ ]:
n.add_edge('c', 'd')
n.add_edge('d', 'a')
print(n)

In [ ]:
'd' in n.nodes

## 1.2 Node and Edge objects <a class="anchor" id="part-1-2"></a>

In the simple example above, we have generated nodes and edges by calling the `add_node` and `add_edge` function of the network instance. Internally, nodes and edges are represented as objects of type `Node` and `Edge` that can be stored in one or multiple instances of type `Network`. Just like a `Network`, each instance of a `Node` and `Edge` has a UID. In the example above, `pathpy` has automatically created `Node` and `Edge` instances and has assigned the UIDs `a`, `b`, `c`, and `d` to those nodes. We can access those node objects via the node container `Network.nodes`. We can iterate through this dictionary to print a summary of all node objects:

In [ ]:
for v in n.nodes:
    print(v)

We can also use the uid of a node to access a specific node object in a network by using the uid as an index to the `nodes` container:

In [ ]:
print(n.nodes['a'])

Importantly, the same node object can be added to multiple networks (e.g. if we want to capture that the same set of nodes is connected via different network topologies).

In [ ]:
n2.add_node(n.nodes['a'])
print(n2)

However, each network can contain only a single node with a given uid, so the following will actually raise an Error:

In [ ]:
try:
    n.add_node('a')
except KeyError:
    pass

Similar to `nodes`, the `edges` container of the network contains all edges of a network and each edge is actually stored as an `Edge` object. Let us iterate through the edges container of network n to better understand this:

In [ ]:
for e in n.edges:
    print('---')
    print(e)

The confirms that the edge container contains one instance of `Edge` for each edge that we added before. Each `Edge` has again a unique identifier, which has been automatically assigned in our example above. Just like for `Node` or `Network` objects, we can manually create an edge object with a custom UID as follows. To create an edge with a different UID that connects node 'd' and 'e' we write:

In [ ]:
n.add_edge('d','e',uid='MyEdge')

We might want to add a different edge between two edges in the same network. The default network object generated by `pathpy` does not allow that

In [ ]:
try:
    n.add_edge('d','e',uid='AnOtherEdge')
except KeyError:
    pass

To do that we have to pass the multi-edge parameter in the creation of the network.  `Network`.

In [ ]:
n_multi = pp.Network(directed=False, uid='ExampleNetwork', multiedges = True)
for e in n.edges:
    n_multi.add_edge(e)

After specifying the multi-edge parameter we can add the edge with a different UID that connects node 'd' and 'e':

In [ ]:
n_multi.add_edge('d','e',uid='AnOtherEdge')

In [ ]:
print(n_multi)

The summary of the network confirms that the network now contains six edges (between five pairs of nodes). This native support for multi-edge networks is an important feature of `pathpy`. It also means that every pair of nodes can be connected by more than one edge. We can access those edges via the `Network.edges` container in multiple ways. First, we can simply iterate through the edge objects as shown before. Second, we can directly access an `Edge` with a given UID as follows:

In [ ]:
print(n_multi.edges['MyEdge'])

Finally, we often want to access those edges that connect a specific pair of nodes. We can thus pass a pair of node uids as index to `Network.edges`. However, since multiple edges between the same pair of nodes are possible, this returns a list of Edge objects, which - in the case of the node pair `a` and `b` contains two elements:

In [ ]:
print(n_multi.edges['d', 'e'])

## 1.3 Attributes <a class="anchor" id="part-1-3"></a>

You may wonder why `pathpy` stores nodes and edges as objects rather than as simple strings or numbers. The reason is that we often want to use networks to model relational data that contains additional information on nodes, edges, or networks. To support this, all `pathpy` objects can store arbitrary key-value data in the form of attributes. Let's explore this in a toy example for a social network:

In [ ]:
trolls = pp.Network(uid='Trolls')
trolls.add_node('t', name= 'Tom',age=156)
trolls.add_node('b', name= 'Bert',age=96)
trolls.add_node('w', name= 'William',age=323)
trolls.add_edge('t', 'b', uid='t-b',strength=2.0)
trolls.add_edge('t', 'w', uid='t-w',strength=5.0)
print(trolls)

Just like nodes, `Edge` objects can store arbitrary attributes that we can read and write via the `edges` dictionary in the `Network` class. We can store attributes as follows:

In [ ]:
trolls.edges['t-b']['type'] = 'like'
trolls.edges['t-w']['type'] = 'dislike'

In [ ]:
print(trolls.edges['t-b'].attributes)
print(trolls.edges['t-w'].attributes)

Numerical properties of edges are often used to store the strength or weight of interactions in a system. Such numerical property can also be considered in the degree calculation. Above, we have a directed network, so we can use the `indegrees()` and `outdegrees()` function to access the directed degrees of nodes. In our example, Tom has an outdegree of two since there are two directed edges to Bert and William:

In [ ]:
trolls.outdegrees()['t']

To additionally consider numerical attributes in the degree calculation, we can use the weight parameter of the degrees(), indegrees() or outdegrees() functions. To calculate the total strength of all outgoing edges from Tom we can write:

In [ ]:
trolls.outdegrees(weight='strength')['t']

## 1.3 Functions <a class="anchor" id="part-1-4"></a>

### Adjacency matrices

Adjacency matrices are an important mathematical representation of networks. The topology of a matrix is represented in terms of a matrix A, where an entry A[i,j]=1 indicates that an edge exists from the i-th to the j-th node of the network. The absence of edges is encoded by zero entries. The size of an adjacency matrix representation of a network with n nodes is generally $n^2$, which is not suitable for networks with thousands or millions of nodes. `pathpy` nevertheless supports efficient adjacency matrix calculation for *sparse* networks, i.e. networks where the majority of node pairs are not connected by an edge. Instead of a fully populated matrix, a call to `Network.adjacency_matrix()` returns a *sparse matrix object*, i.e. an efficient representation of the indices and values of non-zero entries:

In [ ]:
print(trolls.adjacency_matrix())

The adjacency matrix entries above shows that two edges from Tom to Bert and from Tom to William exist. Moreover, the fact that the matrix is assymetric tells us that this is a directed network. By default, a binary matrix representation is returned where entries store the presence or absence of edges as 0 or 1 entries. If we want to use numerical attributes of edges instead, we can again pass the name of a numerical attribute that should be used as edge `weight`:

In [ ]:
print(trolls.adjacency_matrix(weight='strength'))

### Betweenness Centrality

If we want a centrality measure that considers the topology of links (and not only the number of links incident to nodes) we can measure the betweenness centrality.

In [ ]:
trolls.add_edge('a', 't', uid='a-t',strength=1.0)

In [ ]:
trolls.betweenness_centrality()

or via `algorithms`:

In [ ]:
pp.algorithms.centralities.betweenness_centrality(trolls)

# 2 Visualising networks with `pathpy`<a class="anchor" id="part-2"></a>

2.1 [Interactive network visualisation in `jupyter`](#part-2-1)  
2.2 [Network Layouts](#part-2-2)  
2.3 [Plotting spatially embedded networks](#part-2-3)  
2.4 [Plotting networks to HTML, PDF and LaTeX](#part-2-4)

A key feature of `pathpy` is its support for custiumizable interactive visualisations that can be embedded in jupyter notebooks or stored as stand-alone files. In the following, we show this functionality in some toy exammples before moving to real data sets in the next unit. We first import `pathpy` as usual.

## 2.1 Interactive network visualisation in `jupyter` <a class="anchor" id="part-2-1"></a>

We first create a simple toy example by adding two edges between three nodes.

In [ ]:
n = pp.Network(directed=False)
n.add_edges(('a', 'b'), ('b', 'c'))
print(n)

Calling the `print` function on a network instance will generate a string representation that can be printed on the console. The simplest way to graphically visualise the network in a jupyter notebook is to simply type the variable name of the network:

In [ ]:
n

This will create an interactive HTML visualisation of the network, where we can zoom, pan, and drag nodes. The same visualisation is generated if we explicitly call the function `plot` on the network instance. Try to zoom and pan the network (by holding the shift key and using the mouse/mouse wheel). Try what happens if you drag a node and release the mouse button. Try to search for a node with a specific uid or name:

In [ ]:
n.plot()

Using the two top-left buttons in the visualisation you can export the current view of the network as a SVG vector graphics or a PNG pixel graphics file. By default a default style is applied to the network but pathpy allows to fully style the network based on (i) node and edge attributes that are automatically considered in the visualisation, or (ii) a custom style dictionary that can be passed to the plot function. If we want to change the color of nodes we can simply assign a color attribute to the nodes:

In [ ]:
n.nodes['a']['color'] = 'red'
n.nodes['b']['color'] = 'green'
n.nodes['c']['color'] = 'blue'
n.plot()

We can additionally change the size of the nodes as follows:

In [ ]:
n.nodes['a']['size'] = 20
n.nodes['b']['size'] = 10
n.nodes['c']['size'] = 20
n.plot()

An alternative method to style the network is by means of a key-value arguments with node and edge properties that are passed to the plot function. We can either assign the same property to all nodes, or we can assign different properties to different nodes based on a dictionary:

In [ ]:
n.plot(node_color='red', node_size=10)

A better way to manage plot styles is to store all styles in a dictionary, whose entries are then passed to the plot function using the kwargs operator **. A major advantage of this is that we can store the plot style and apply the same style to multiple networks or to visualisations in multiple formats.

In [ ]:
plot_style = {
    'node_color': {'a': 'orange', 'b': 'blue', 'c': 'red'}, 
    'node_size': {'a': 30, 'b': 10, 'c': 20},
}
n.plot(**plot_style)

In any case, the plot parameters passed explicitly to the plot function will take preference over any node or edge properties. To conclude this section, let us plot a few networks that we read from our database file.

In [ ]:
# n = pp.io.sql.read_network(filename='networks.db', table='gentoo', directed=True)
# n.plot(node_size=5,label_enabled=False,edge_color='gray')

## 2.2 Network Layouts <a class="anchor" id="part-2-2"></a>

An important question in the drawing of networks is where nodes are placed. The positioning of nodes determines whether it is easy to spot patterns in the topology of the network. As a general rule, good network visualisations should have a small number of crossing edges and nodes should be placed away from each other such that we can easily distinguish them. If we make no other effort, pathpy will automatically use a simple, interactive force-directed layout algorithm. It is based on the idea that nodes connected by an edge are moved towards each other by an attractive force, while an additional repulsive force between all node pairs makes sure that nodes are not drawn on top pf each other. The simulation of those forces, and the computation of a steady-state of the resulting many-particle system is what determines the node positions in the default pathpy visualisation. Also, this is the basis upon which nodes move if you disturb the visualisation by dragging around nodes.

While the default layout algorithm makes it simple to visualise networks, it has the disadvantage that the layout is actually calculated in JavaScript. This means that the positions of nodes are actually not stored in python, which makes it impossible to influence (or store) node positions from python. To solve this issue, `pathpy` provides a `layout` module that can be used to precompute node positions based on different layout algorithms. Moreover, it allows to manually optimise the parameters of those algorithms to generate a nice visualisation.

The styling of node positions via a layout style follows the same idea like the styling of plots via a plot style. We can store the parameters in a dictionary and pass them to the `layout` function using the ** operator. To compute ode positions based on a certain number of iterations of the Fruchterman-Reingold algorithm with a specific force value we can write:

In [ ]:
layout_style = {}
layout_style["node_size"] = 2
layout_style['layout'] = 'Fruchterman-Reingold'
layout_style['force'] = 0.2
layout_style['iterations'] = 500
layout = pp.layout(n, **layout_style)
print(layout)

This computes a dictionary of node positions, where the node uids are the keys and two-dimensional coordinates are the values. We can now pass this specific layout to the plot function of the network. This will disable the interactive layout in JavaScript, fixing the node positions to the precalculated layout:

In [ ]:
plot_style = {
    'node_size': 5,
    'edge_color': "grey",
    'label_enabled':False
}


pp.plot(n, **plot_style, layout=layout)

## 2.3 Plotting spatially embedded networks <a class="anchor" id="part-2-3"></a>

Layout algorithms are important to visualise networks where nodes are not naturally embedded in a space. However, for a number of complex networks we can naturally assign coordinates to nodes. Examples include infrastructure networks like road, train, or flight networks where we have information on geographic coordinates of road junctions, train stations, or airports. We do not need a layout algorithm to calculate node positions, we can use node coordinates instead, which we can assign to the node attributes:

In [ ]:
n = pp.Network() 

n.add_node("a",coordinates=[0,0],color='orange',size=30)
n.add_node("b",coordinates=[0,1],color='blue',size=10)
n.add_node("c",coordinates=[1,0],color='red',size=20)

n.add_edge("a", "b")
n.add_edge("b", "c")

plot_style = {
}
n.plot(**plot_style)

## 2.4 Plotting networks to HTML, PDF and LaTeX <a class="anchor" id="part-2-4"></a>

While it is convenient to interactively plot networks in a jupyter network, we often want to generate stand-alone visualisations that we can either share or embed in scientific publications. Apart from the possibility to directly export PNG and SVG visualisations from the interactive HTML widget, the plot function can be used to generate a stand-alone HTML visualisation of the network that can be opened in any browser and shared on the Web.

In [ ]:
n.plot( filename='network.html', **plot_style) 

If you want a vector graphics figure that can be embedded in a scientific paper, you can use the plot function to export network visualisations as PDF:

In [ ]:
n.plot(filename='network.pdf',**plot_style)

An interesting feature of pathpy is that it builds on the package `tikz-network`, a powerful framework to draw graphs in LaTeX. If we want to generate a visualisation in which we retain the full power to style the network in LaTeX, we can export the network as a tex file that can be build with a LaTeX compiler:

In [ ]:
n.plot(filename='network.tex', **plot_style)

# 3 Temporal network analysis and visualisation in `pathpy`<a class="anchor" id="part-3"></a>

3.1 [Creating temporal networks](#part-3-1)  
3.2 [Calculating causal paths](#part-3-2)  
3.3 [Time-aggregated networks](#part-3-3)  
3.4 [Visualization of temporal networks](#part-3-4)

## 3.1 Creating temporal networks <a class="anchor" id="part-3-1"></a>

`pathpy` provides special support for the analysis of temporal networks data via its `TemporalNetwork` class. It is suitable for data that captures time-stamped edges  `(v,w,t)`  instantaneously occurring at discrete time stamps  `t`. Let us start by creating an empty instance of this class.

In [ ]:
t = pp.TemporalNetwork()
print(t)

Let us add some time-stamped edges to this instance.

In [ ]:
t.add_edge('a', 'b', t=1)
t.add_edge('b', 'a', t=3)
t.add_edge('b', 'c', t=3)
t.add_edge('d', 'c', t=4)
t.add_edge('c', 'd', t=5)
t.add_edge('c', 'b', t=6)
print(t)

We get basic summary statistics, such as the number of time-stamped interactions, the minimum and maximum timestamp, the duration of the observation, the number of different timestamps, as well as the average, minimum, and maximum time difference between consecutive time-stamped edges.

Just like other pathpy objects, we can directly embed interactive visualisations of a TemporalNetwork in-line in jupyter. Let us try this with our first example instance t.

In [ ]:
t

## 3.2 Calculating causal paths <a class="anchor" id="part-3-2"></a>

In [ ]:
dag = pp.DirectedAcyclicGraph.from_temporal_network(t,delta=1)
print(dag)

In [ ]:
node_mapping = {node.uid:node['original'].uid for node in dag.nodes}
print(node_mapping)

In [ ]:
colors = {'a': 'red', 'b': 'blue', 'c': 'cyan', 'd': 'orange'}
node_colors = { v: colors[node_mapping[v]] for v in node_mapping }
dag.plot(node_color=node_colors)

## 3.3 Time-aggregated networks <a class="anchor" id="part-3-3"></a>

In [ ]:
pp.Network.from_temporal_network(t)

In [ ]:
p = t.to_paths()
pp.Network.from_paths(p)

## 3.4 Visualization of temporal networks <a class="anchor" id="part-3-4"></a>

We get a similar plot as observed by the static networks, however with an additional menu bar and time slider. When we click play, nothing happens since we only consider one temporal edge. To see some changes we have to add another edge.


In [ ]:
t = pp.TemporalNetwork(directed=False)
t.add_node('a',color='red',size=10)
t.add_node('b',color='blue',size=15)
t.add_node('c',color='pink',size=12)
t.add_edge('a','b',color='green',size=4,begin=0,end=10)
t.add_edge('b','c',begin=7,end=15)
# t

In [ ]:
t.nodes['a'].update(color='green',size=20,t=5)
t.edges['a','b'].update(color='gray',size=2,t=4)
t

# 4 Higher-order models of paths <a class="anchor" id="part-4"></a>

4.1 [Path and PathCollection](#part-4-1)  
4.2 [Creating higher-order networks](#part-4-2)  
4.3 [Ranking nodes in higher-order networks](#part-4-3)  
4.4 [Multi-order model selection](#part-4-4)

So far, we have focused on network models, but the real **purpose of `pathpy` is to fit and analyse higher-order models for paths in complex networks**. We will delve in higher order models in a moment in a moment, first we are going to focus on paths, the basic building block for the construction of Higher Order Networks.

## 4.1 Path and PathCollection <a class="anchor" id="part-4-1"></a>

Like nodes and edges, in pathpy also paths are represented through objects using the `path` class.
A path is a collection of consecutive edge objects and can be created as follows.

In [ ]:
a = pp.Node("a")
b = pp.Node("b")
c = pp.Node("c")

ab = pp.Edge(a,b,uid='a-b')
bc = pp.Edge(b,c,uid='b-c')

p = pp.Path(ab,bc,uid='a-b-c')
print(p)

Multiple paths can be grouped in a `PathCollection` object, this represents observed transitions (sequences) over the nodes of a network.

In [ ]:
paths = pp.PathCollection()
paths.add(p, frequency = 20)

We can inspect the Path objects contained in a path collection

In [ ]:
for path in paths:
    print(path.uid, ":" , path.nodes)

We can access the path objects in the path collection through their uid, nodes or edges

In [ ]:
print(paths['a-b-c'],'\n')
print(paths['a','b','c'],'\n')
print(paths['a-b','b-c'])

The method outlined above was rather cumbersome. Indeed Path collections can be created more conveniently by directly passing paths as sequences of node uids to the path collection object

In [ ]:
paths = pp.PathCollection()
paths.add('a','c','d',uid='acd',frequency=10)
paths.add('b','c','e',uid='bce',frequency=10)
print(paths)

## 4.2 Creating higher-order networks <a class="anchor" id="part-4-2"></a>

From a higher-order network analytic point of view, standard graphs or networks are **first-order probabilistic generative models** for paths in complex networks. As we have seen in 1.2, they can be viewed as **maximum entropy models that consider first-order dyad statistics** (i.e. edge frequencies), while ignoring higher-order dependencies in the real-world path, sequence, or time-series data.

In the following works, we have studied measures for higher-order correlations in such data and we generalised network models to higher-order models with arbitrary order:

- R Pfitzner, I Scholtes, A Garas, CJ Tessone, F Schweitzer: **Betweenness Preference: Quantifying Correlations in the Topological Dynamics of Temporal Networks**, In Physical Review Letters, May 2013, [arXiv 1208.0588](http://arxiv.org/abs/1208.0588)

- I Scholtes, N Wider, R Pfitzner, A Garas, CJ Tessone, F Schweitzer: **Causality-driven slow-down and speed-up of diffusion in non-Markovian temporal networks**, In Nature Communications, September 2014, [arXiv 1307.4030](http://arxiv.org/abs/1307.4030)

- I Scholtes, N Wider, A Garas: **Higher-Order Aggregate Networks in the Analysis of Temporal Networks: Path structures and centralities**, In The European Physical Journal B, March 2016, [arXiv 1508.06467](http://arxiv.org/abs/1508.06467) 

- I Scholtes: **When is a Network a Network? Multi-Order Graphical Model Selection in Pathways and Temporal Networks**, In KDD'17, February 2017, [arXiv 1702.05499](https://arxiv.org/abs/1702.05499)

A broader overview of the research on higher-order models for complex systems is available in the following article:

- R Lambiotte, M Rosvall, I Scholtes: **From Networks to Optimal Higher-Order Models of Complex Systems**, Nature Physics, March 2019,  [arXiv 1806.05977](https://arxiv.org/abs/1806.05977)

The data analysis and modelling framework outlined in these works build on a generalisation of standard, first-order networks to $k$-dimensional De Bruijn graph models for paths in complex networks.

The class `HigherOrderNetwork` allows us to generate such higher-order network models of paths. In the documentation, we find that the constructor takes a parameter `network`, i.e. the network with the observed paths we want to model. With the setting `order` we specify the order of the higher-order model that we want to fit. To understand this better, let us do this for our toy example.

In [ ]:
hon_1 = pp.HigherOrderNetwork.from_paths(paths,order=1)
print(hon_1)

This generates a first-order model of our paths, with five nodes `a`,`b`,`c`,`d` and `e`, and four links `a-c`, `b-c`, `c-d` and `c-e`. It is identically to the `Network` instance that we have previously created. Indeed, each `HigherOrderNetwork` instance is derived from the class `Network`, which means we can store edge and node attributes and visualise it by precisely the same methods.

In [ ]:
hon_1

In [ ]:
for edge in hon_1.edges:
    print(edge,edge['frequency'])

This output confirms that a `HigherOrderModel` with `order=1` is identical to our `Network` model. We can see this network as a **first-order model** for paths where **edges are paths of length one**. That is, in a model with order `order=1` edge weights capture the statistics of paths of `length=1`.

We can generalise this idea to **k-th-order models** for paths, where **nodes are paths of length $k-1$** while **edge weights capture the statistics of paths of length $k$**. We can generate such a $k$-th order model by performing a line graph transformation on a model with order $k-1$. That is, edges in the model of order $k-1$ become nodes in the model with order $k$. We then draw edges between higher-order nodes whenever there is a possible path of length $k$ in the underlying network. The result is a $k$-dimensional De Bruijn graph model for paths. Let us try this in our example.

In [ ]:
#hon_2 = pp.HigherOrderNetwork.from_paths(paths,order=2)
hon_2 = pp.HigherOrderNetwork(order=2)
hon_2.fit(paths)
print(hon_2)

In [ ]:
hon_2

In [ ]:
for edge in hon_2.edges:
    print(edge,edge['frequency'])

Each of the four **edges** in the first-order model is now represented by a **node** in the second-order model. We further have two directed edges `a-c-d` and `b-c-e` that represent the two paths of length two that occur in our data.

This is important because it captures to what extent the paths that we observe in our data deviate from what we would expect based on the (first-order) network topology of the system. Considering such a first-order model, all four paths `a-c-d`, `a-c-e`,`b-c-d`, and `b-c-e` of length two are possible. If edges were statistically independent, we would expect those four paths to occur with the same frequency.

Another way to express this independence assumption is to consider Markov chain models for the sequences of nodes traversed by a path. In this view, independently occurring edges translate to a memoryless first-order Markov process for the node sequence. In our example, we expect paths `a-c-d` and `a-c-e` to occur with the same probability, i.e. the next nodes `d` or `e` on a path through `c` are independent of the previous node `a`, their probabilities only depending on the relative frequency of edges `c-d` vs `c-e`. In our toy example, we have a total of 20 observed paths of length two, so we expect each of the path to occur five times on average.

`pathpy` can actually generate this **null-model** for paths within the space of possible second-order models. This allows us to compare how the observed path statistics deviate from a (Markovian) expectation.

In [ ]:
hon_2_null = pp.NullModel.from_paths(paths,order=2)

In [ ]:
for edge in hon_2_null.edges:
    print(edge,edge['frequency'])

## 4.3 Ranking nodes in higher-order networks <a class="anchor" id="part-4-3"></a>

Higher-order models capture deviations from the transitivity assumption that we often implicitly make when we apply standard graph mining or network analysis techniques. An important class of methods that rely on this assumption are centrality measures, which are often used to rank nodes in networked systems.

Let us consider the betweenness value of a node $v$, which (in its unnormalized variant) captures for how many pairs of nodes $x,y$ the shortest path from $x$ to $y$ passes through $v$. This measure is important, because is teaches us something about the role of nodes in information flow. It further captures to what extent removing a node will affect the shortest paths between other nodes. Let us try this in our example:

In [ ]:
pp.algorithms.betweenness_centrality(hon_1)['c']

This is easy to understand, as there are four pairs `b-e`, `b-d`, `a-e`, and `a-d` for which the shortest path in the topology passes through node `c`. However, this notion of *importance* of node `c` is based on the assumption that paths are transitive and Markovian, which is not the case in our observed paths. `pathpy` actually allows us to calculate the betweenness of node `c` in the observed paths `paths`.

In [ ]:
pp.algorithms.betweenness_centrality(paths)['c']

Not surprisingly, the betweenness of node $c$ in the actual paths is two, as $c$ only connects two pairs of nodes are connected via a (shortest) path. This change in centrality is captured in the higher-order model and `pathpy` allows us to directly generalize centrality measures to higher orders, simply by calling the functions on the class `HigherOrderNetwork`.

In [ ]:
pp.algorithms.betweenness_centrality(hon_2)['c']

We see that in this example (since we **only** have second-order dependencies) the second-order model accurately captures the betweenness of node $c$. Let us contrast this to the second-order null model.

In [ ]:
pp.algorithms.betweenness_centrality(hon_2_null)['c']

This is what we expect, since this null model is simply a second-order representation of the first-order model, in which paths are transitive and memoryless.

## 4.4 Multi-order model selection <a class="anchor" id="part-4-4"></a>

So far, we studied higher-order network models for path data with a fixed, given order $k$. We have seen that such higher-order models can yield better predictions compared to standard network models. However, a critical question arises: how can we decide **at which order we should model a given system**? This points to a more general problem as we can also imagine systems for which path statistics do not deviate significantly from the transitive, Markovian assumption made by a first-order model. So we **need methods to decide when higher-order models are actually required**.

Moreover, a higher-order model with order $k$ can only capture higher-order dependencies at a single fixed correlation length $k$. But we may encounter data that exhibit multiple correlation lengths at once. How can we **combine models with multiple higher orders into a multi-order model**?

To fix the issues above, we need a probabilistic generative model that can deal with extensive collections of (short) paths in a network. The key idea is to combine multiple higher-order network models into a single multi-layered, multi-order model. To calculate the likelihood of such a model, we can use all layers, thus avoiding the problem that we discard prefixes of paths. For each path, we start the calculation at a layer of order zero, which considers the relative probabilities of nodes. We then use this model layer to calculate the probability to observe the first node on a path. For the next transition to step two, we then use a first-order model. The next transition is calculated in the second-order model and so on until we have reached the maximum order of our multi-order model. At this point, we can transitively calculate the likelihood based on the remaining transitions of the path.

The method is described in all details in the following paper:

I Scholtes: [When is a Network a Network? Multi-Order Graphical Model Selection in Pathways and Temporal Networks](http://dl.acm.org/citation.cfm?id=3098145), In KDD'17 - Proceedings of the 23rd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining, Halifax, Nova Scotia, Canada, August 13-17, 2017

But let us go to practice. `pathpy` can directly generate and analyse multi-order network models. Let us try this in our example.

In [ ]:
mom = pp.MultiOrderModel.from_paths(paths,max_order=2)
print(mom)

Rather than performing a likelihood test ourselves, we can actually simply call the method `MultiOrderModel.predict`. It will return the maximum order among all of its layers for which the likelihood ratio test rejects the null hypothesis.

In [ ]:
mom.predict()

We now test whether this approach to **learn the optimal representation of path data** actually works. For this, let us generate path statistics that are in line with what we expect based on a first-order network model, and check whether the order estimation gives the right result.

In [ ]:
random = pp.PathCollection()
random.add('a','c','d',frequency=5)
random.add('a','c','e',frequency=5)
random.add('b','c','d',frequency=5)
random.add('b','c','e',frequency=5)

mom_2 = pp.MultiOrderModel.from_paths(random,max_order=2)
mom_2.predict()

# 5 Outlook <a class="anchor" id="part-5"></a>

5.1 [Hypergraphs](#part-5-1)  
5.2 [Milestones](#part-5-2)  
5.3 [Your contribution](#part-5-3)  

# 5.1 Hypergraphs <a class="anchor" id="part-5-1"></a>

In [ ]:
a = pp.Node('a')
b = pp.Node('b')
c = pp.Node('c')
d = pp.Node('d')

e = pp.HyperEdge({a, b}, {c, d}, uid='ab-cd')
print(e)

# 5.2 Milestones <a class="anchor" id="part-5-2"></a>

<div class="alert alert-info">
The first Beta version of pathpy is expected to be releast at the end of 2020. 
</div>

- Documentation
- Performance
- Online tutorials
- Visualisations

# 5.3 Ways to contribute <a class="anchor" id="part-5-3"></a>

There are many ways to contribute to `pathpy`, with the most common ones being contribution of code or documentation to the project. Improving the documentation is no less important than improving the library itself. If you find a typo in the documentation, or have made improvements, do not hesitate to send an email to the mailing list or preferably submit a GitHub pull request.

Another way to contribute is to report issues you’re facing, and give a “thumbs up” on issues that others reported and that are relevant to you. It also helps us if you spread the word: reference the project from your blog and articles, link to it from your website, or simply star to say “I use it”:

- Submitting a bug report or a feature request
- Contributing code
- Documentation